In [26]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
# from sklearn.model_selection import train_test_split

# Load the dataset from an Excel file
kmp = pd.read_excel('MusicKeen.xlsx' )
X = kmp.drop(columns=['genre'])
y =kmp['genre']

# Define features
numerical_features = ['age']
categorical_features = ['gender', 'obsession']

# Ordinal features with their categories
ordinal_features = {
    'obsession': ['Low', 'Moderate', 'High'],
}

# Make a copy of X for encoding
X_encoded = X.copy()

# Create and store ordinal encoders for each ordinal feature
ordinal_encoders = {}
for feature, categories in ordinal_features.items():
    encoder = OrdinalEncoder(categories=[categories])
    X_encoded[feature] = encoder.fit_transform(X[[feature]])
    ordinal_encoders[feature] = encoder

# Remaining categorical features will be one-hot encoded
remaining_categoricals = [f for f in categorical_features if f not in ordinal_features]

# Create preprocessing pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), remaining_categoricals),
        ('ord', 'passthrough', list(ordinal_features.keys()))
    ]
)

# Encode y if needed
if y.dtype == 'object':
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
else:
    y_encoded = y

# Fit the preprocessor and transform X
X_transformed = preprocessor.fit_transform(X_encoded)

# Train model
model = DecisionTreeClassifier()
model.fit(X_transformed, y_encoded)

# Create sample data for prediction
sample_data = {
    'age': 40,
    'gender': 'Female',
    'obsession': 'Moderate'
}

sample = pd.DataFrame([sample_data])

# Encode the sample data using the same preprocessing steps
sample_encoded = sample.copy()

# Use the same ordinal encoders that were used for training
for feature, encoder in ordinal_encoders.items():
    sample_encoded[feature] = encoder.transform(sample[[feature]])  

sample_transformed = preprocessor.transform(sample_encoded)

# Make prediction
predictions_encoded = model.predict(sample_transformed)

# Decode the predictions if necessary
if y.dtype == 'object':
    predictions = le.inverse_transform(predictions_encoded) 
else:
    predictions = predictions_encoded

print(f"Predicted genre for the sample data: {predictions[0]}")

Predicted genre for the sample data: Classical
